In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import torch
from networks import networks
from PIL import Image
from utils import show, renormalize, compositions, masking, imutil
import matplotlib.pyplot as plt
import numpy as np
import os

### load the GAN, encoder, and part extractor

In [ ]:
gtype = 'stylegan'
domain = 'car'
nets = networks.define_nets(gtype, domain)
compositer = compositions.get_compositer(domain)(nets)

### visualize the reconstruction, and masked reconstruction

In [ ]:
with torch.no_grad():
    im = nets.seed2image(1,seed=10)
    show.a(['Input Image', renormalize.as_image(im[0]).resize((256, 256), Image.ANTIALIAS)])
    rec = nets.invert(im, mask=None)
    show.a(['Reconstructed', renormalize.as_image(rec[0]).resize((256, 256), Image.ANTIALIAS)])
    show.flush()
    hints, mask = masking.mask_upsample(im, mask_cent=0.5 if gtype == 'proggan' else 0.)
    rec_mask = nets.invert(hints, mask=mask)
    show.a(['Input Masked', renormalize.as_image(hints[0]).resize((256, 256), Image.ANTIALIAS)])
    show.a(['Rec. Masked', renormalize.as_image(rec_mask[0]).resize((256, 256), Image.ANTIALIAS)])
    show.flush()

### randomly sample image parts, and compose them

In [ ]:
rng = np.random.RandomState(0)
indices = rng.choice(compositer.total_samples,
                     len(compositer.ordered_labels))
with torch.no_grad():
    composite_data = compositer(indices)

In [ ]:
for im, m in zip(composite_data.parts_image, composite_data.parts_mask):
    show.a([renormalize.as_image(im).resize((128, 128), Image.ANTIALIAS)], cols=6)
    a, b, c = imutil.draw_masked_image(im[None], m[None], size=128)
    show.a([c], cols=6)
show.flush()

In [ ]:
show.a(['Network Input', renormalize.as_image(composite_data.composite_image[0]).resize((256, 256), Image.ANTIALIAS)])
show.a(['Network Output', renormalize.as_image(composite_data.inverted_RGBM[0]).resize((256, 256), Image.ANTIALIAS)])
show.flush()